In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from ast import literal_eval
from math import log10, log

from IPython.display import Markdown, display, HTML
import pandas as pd

past_queries = ['criativo','Instagram','reprodução','real','hostil']

# Query Expansion

<br>

In this report we'll apply and compare association metrics to help us expand queries made in a previous report. Employed data and auxiliary scripts can be found at the report's [original repository](https://github.com/Benardi/bochica).

<br>

In [3]:
inv_index = pd.read_csv("../output/inverted_index.csv").set_index(['word'])\
            .drop('vivoÉ').drop('ônibusFoi') #remove nltk tokenization mistake
inv_index["doc_id:freq"] = inv_index["doc_id:freq"].apply(lambda x: literal_eval(x))
display(Markdown("## Inverted Index"))
inv_index.head(5)

## Inverted Index

,doc_id:freq
word,
juíza,"[(0, 2), (1, 1)]"
federal,"[(0, 1), (1, 1), (2, 1), (6, 2), (14, 1), (36,..."
Ivani,"[(0, 1), (1, 1)]"
Silva,"[(0, 3), (1, 1), (5, 1), (13, 2), (25, 1), (72..."
Luz,"[(0, 3), (1, 1), (124, 1)]"


***

<br>

## Best metric to find strongly associated word

* Let's apply the process below to the 5 single word queries of last report:
    * `Instagram`
    * `reprodução`
    * `criativo`
    * `hostil`
    * `real`


Let's apply four different metrics:
* `Dice’s Coefficient`
* `Pearson’s Chi-squared`
* `Mutual Information Measure (MIM)`
* `Expected Mutual Information Measure (EMIM)`


For each metric we shall choose the top 10 most strongly associated words. 

    
> Comparing the results we shall choose the best metric

In [4]:
def mim(**kwargs):
    """ Computes the Mutual Information Measure score.

    Calculates the association between two terms using
    the Mutual Information Measure score.
    
    :param int n_a: number of matches for term A
    :param int n_b: number of matches for term B
    :param int n_ab: number of matches for term A and B
    
    :return: MIM score 

    :rtype: number
    """
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]
    
    return n_ab/(n_a * n_b)

def dice_coef(**kwargs):
    """ Computes the Dice's coefficient.

    Calculates the association between two terms using
    the Mutual Information Measure score.
    
    :param int n_a: number of matches for term A
    :param int n_b: number of matches for term B
    :param int n_ab: number of matches for term A and B
    
    :return: Dice's coefficient 

    :rtype: number
    """
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]

    return n_ab/(n_a + n_b)

def emim(**kwargs):
    """ Computes the Expected Mutual Information Measure 
    score.

    Calculates the association between two terms using
    the Expected Mutual Information Measure score.
    
    :param int n_a: number of matches for term A
    :param int n_b: number of matches for term B
    :param int n_ab: number of matches for term A and B
    :param int N: total number of terms
    
    :return: Expected Mutual Information Measure score 

    :rtype: number
    """
    N = 10e6 if 'N' not in kwargs else kwargs["N"]
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]
    result = 0 if n_ab == 0 else \
             n_ab * log10(N * (n_ab / (n_a * n_b)))
    
    return result

def chi_square(**kwargs):
    """ Computes the Pearson’s Chi-squared score.

    Calculates the association between two terms using
    the Pearson’s Chi-squared score.
    
    :param int n_a: number of matches for term A
    :param int n_b: number of matches for term B
    :param int n_ab: number of matches for term A and B
    :param int N: total number of terms
    
    :return: Expected Pearson’s Chi-squared score 

    :rtype: number
    """
    N = 10e6 if 'N' not in kwargs else kwargs["N"]
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]

    return pow((n_ab - (1/N) * n_a * n_b),2) / (n_a * n_b)

In [5]:
from bisect import insort_left


def n_most_related(n=10,**kwargs):
    metric = kwargs.pop("metric")
    index = kwargs.pop("df")
    A = kwargs.pop("A")  
    words = index.drop(A).reset_index()["word"].values
    rank = []
    
    for word in words:
        n_a = index.xs(A).values[0]
        n_b = index.xs(word).values[0]
        n_ab = [val1 for (val1,val2) in n_a if val1 \
            in (val1 for (val1,val2) in n_b)]
        kwargs["n_ab"] = len(n_ab)
        kwargs["n_a"] = len(n_a)
        kwargs["n_b"] = len(n_b)
        insort_left(rank, (metric(**kwargs),word))
        rank = rank[-n:]
    
    return rank

In [6]:
# number of documents in the collection
n_docs = inv_index["doc_id:freq"].apply(lambda x: list(i[0] for i in x)).sum()
n_docs = len(set(n_docs))
display(Markdown("* Let's employ the number  of documents in this particular \
                  collection (N={}) in our calculations".format(n_docs)))

* Let's employ the number  of documents in this particular                   collection (N=249) in our calculations

In [7]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='Instagram',
                            metric=metric,n=10, N=n_docs)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("### Top 10 words most associated with 'Instagram' by metric"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns=columns).to_html(index=False)))

### Top 10 words most associated with 'Instagram' by metric

MIM,EMIM,Chi-Squared,Dice
êxito,publicadas,publicadas,publicadas
ânsias,contínuos,contínuos,contínuos
votarão,Gallup,Gallup,Gallup
votará,sociais,tarifas,tarifas
vinhos,tarifas,exibe,exibe
viatura,exibe,êxito,jogam
velinhas,aparece,ânsias,império
utópico,totalmente,votarão,gás
usos,torna,votará,aço
urbana/rural,jogam,vinhos,encerrou


* *EMIM* seems to have the best results, as they have more to do with the term *Instagram*

In [8]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='criativo',
                            metric=metric,n=10, N=n_docs)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("### Results for 'criativo'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))

### Results for 'criativo'

MIM,EMIM,Chi-Squared,Dice
ângulos,cheque,ângulos,ângulos
virarem,conhecidas,virarem,virarem
violou,acredito,violou,violou
viciado,mental,viciado,viciado
vestiam,trabalhadores,vestiam,vestiam
veste,parlamentares,veste,veste
vermelho-sangue,efeitos,vermelho-sangue,vermelho-sangue
vaidades,cara,vaidades,vaidades
vaginas,podemos,vaginas,vaginas
usarem,acontecer,usarem,usarem


* *EMIM* is the only algorithm whose result differs from the rest, being the less coherent at that.

In [9]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='reprodução',
                            metric=metric,n=10, N=n_docs)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("### Results for 'reprodução'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))

### Results for 'reprodução'

MIM,EMIM,Chi-Squared,Dice
útero,plataforma,médicas,médicas
úteis,programas,costumes,costumes
ária,médicas,útero,vazio
workshops,costumes,úteis,valem
votam,música,ária,usuário
vivências,vazio,workshops,preparação
visualizações,valem,votam,bruto
virgindade,usuário,vivências,oferecem
verbalmente,preparação,visualizações,influentes
vendedor,bruto,virgindade,experiências


* Chi-Squared seems to have the best results 

In [10]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='real',
                            metric=metric,n=10, N=n_docs)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("### Results for 'real'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))

### Results for 'real'

MIM,EMIM,Chi-Squared,Dice
úteis,significa,justamente,significa
ítalo-americano,justamente,salto,justamente
ítalo-americana,realidade,estranhas,mostrar
índole,ficção,cristão,ficção
êxito,mostrar,animação,realidade
éticas,perder,Cohen,perder
ânus,acreditam,inédito,chama
ânsias,salto,ficção,acreditam
ávido,chama,acreditam,efeito
áspero,vida,sentar,conteúdo


* *EMIM* seems to have the best results, as in the more coherent with the term *real*

In [11]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='hostil',
                            metric=metric,n=10, N=n_docs)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("### Results for 'hostil'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))

### Results for 'hostil'

MIM,EMIM,Chi-Squared,Dice
utensílios,fere,fere,fere
tornasse,delicado,utensílios,utensílios
tensionamento,judiciais,tornasse,tornasse
suspendendo,Escola,tensionamento,tensionamento
surfar,Chegou,suspendendo,suspendendo
supermovimentada,defendeu,surfar,surfar
situação-limite,causas,supermovimentada,supermovimentada
servidora,posse,situação-limite,situação-limite
rosas,Militar,servidora,servidora
ribeirinhas,sente,rosas,rosas


* *EMIM* seems to have the best results, as in the more coherent with the term *hostil*
    + *delicado* is a notable exception, still overall the result is maintained

> Overall **EMIM** seems to be the best choice, for its results are closer in terms of semantics to the original terms (The results are more coherent)

<br>

***

<br>

# Expading Queries

In [12]:
source = pd.read_csv("../output/results.csv")
source.head()

,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


## Metrics

We shall use the metrics `recall` and `precision` to evaluate the quality of the results once we expand the queries

In [13]:
def recall_score(true_docs, predicted):
    """ Computes the recall score.
    
    Computes the recall score from a list of true documents
    and retrieved ones.
    
    :param List[int] true_docs : list of truly relevant documents.
    :param List[int] predicted :list of documents predicted as relevant

    :return: recall score 

    :rtype: number
    """
    
    true_p = [value for value in predicted if value in true_docs] 
    false_n = list(set(true_docs) - set(predicted))
    return len(true_p) / (len(true_p) + len(false_n))

def precision_score(true_docs, predicted):
    """ Computes the precision score.
    
    Computes the recall score from a list of true documents
    and retrieved ones.
    
    :param List[int] true_docs : list of truly relevant documents.
    :param List[int] predicted :list of documents predicted as relevant

    :return: precision score 

    :rtype: number
    """
    true_p = [value for value in predicted if value in true_docs] 
    false_p = list(set(predicted) - set(true_docs))
    return len(true_p) / (len(true_p) + len(false_p))

## Term At A Time Retrieval

In [14]:
def conjunctive_retrieval_by_term(index, query, k):
    """ Retrieve k matched documents sorted by score.

    Tranverses an inverted index term by term guided by a query 
    to retrieve k documents that contain all query terms 
    in decrescent order of score.
    
    :param pandas.core.frame.DataFrame index: inverted index.
    :param List[string] query: query terms
    :param int k: maximum result length .

    :return: list of documents sorted by score [(score,doc_id)] 

    :rtype: List[(int,int)]
    """
    
    try:
        import Queue as Q  # ver. < 3.0
    except ImportError:
        import queue as Q

    A = {}
    result = []
    R = Q.PriorityQueue()
    q = Q.PriorityQueue()
    L = index.loc[lambda df: df.word.isin(query)]

    for idx, row in enumerate(L.itertuples()):

        if idx == 0:
            for doc_id, freq in getattr(row, "_2"):
                A[doc_id] = freq        
        else:
            for doc_id, freq in getattr(row, "_2"):
                if doc_id not in A:
                    pass
                else:
                    A[doc_id] += freq        

    for doc_id, score in A.items():
        q.put(((-1) * score,doc_id))    

    i = 0
    while not q.empty() and i < k:
        pair = q.get()
        pair = ((-1) *pair[0],pair[1])
        result.append(pair)
        i += 1

    return result

## Instagram (Expanded Queries)

In [15]:
k = 10
term = 'Instagram'

In [16]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10, N=n_docs)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str([term]), str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))

#### Result of *conjunctive retrieve by term* for 'Instagram'

query,top_n_plus,documents
[Instagram],0,"[85, 110, 151, 229, 242]"
"[Instagram, publicadas, contínuos, Gallup]",3,"[85, 151]"
"[Instagram, publicadas, contínuos, Gallup, soc...",5,"[210, 85, 151, 213, 150, 18, 19, 35, 204, 228]"
"[Instagram, publicadas, contínuos, Gallup, soc...",10,"[85, 151, 210, 229, 35, 110, 150, 213, 124, 18]"


In [17]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

,title,subtitle
85,Presidente na ficção humorista é favorito em ...,Admirador declarado de Bolsonaro o ator Volod...
110,Bolsonaro se despede de Israel com promessas e...,Presidente encerrou visita a aliado Netanyahu ...
151,Socialismo ‘millennial’ nos EUA,A temida palavra ‘socialista’ já não é mais ta...
229,Atos pró e contra ditadura militar terminam em...,Grupos se atacaram com paus de bandeira socos...
242,A fortuna blindada de Cristiano Ronaldo,Atacante criou um império empresarial em cuja ...


Above we have a list of the documents that mention the term *Instagram* directly

<br>

In [18]:
pd.set_option('max_colwidth', 200)
# pd.reset_option('^display.', silent=True)
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

,title,subtitle,url
35,Finlândia conclui experiência de renda básica universal com resultados ambíguos,Durante dois anos 2.000 pessoas desempregadas receberam 2.370 reais por mês. Saúde autoestima e otimismo dos beneficiários melhoraram mas situação profissional pouco mudou,https://brasil.elpais.com/brasil/2019/02/09/economia/1549710265_204922.html
228,Quem pagou pelo vídeo revisionista da ditadura distribuído pelo Governo Bolsonaro?,“É preciso saber se foi pago com recursos públicos” diz Gil Castello Branco do Contas Abertas. Ao 'Globo' ator disse ter sido pago. Mourão disse que ideia foi de Bolsonaro,https://brasil.elpais.com/brasil/2019/04/01/politica/1554151562_942843.html
229,Atos pró e contra ditadura militar terminam em confrontos em São Paulo,Grupos se atacaram com paus de bandeira socos e chutes na avenida Paulista neste domingo,https://brasil.elpais.com/brasil/2019/04/01/politica/1554118634_296372.html
204,A guerra de todos contra todos no Governo Bolsonaro,Com chamado aberto à comemoração do golpe de 1964 presidente dobra sua aposta na polarização política,https://brasil.elpais.com/brasil/2019/03/26/politica/1553557825_337887.html
110,Bolsonaro se despede de Israel com promessas econômicas e gafe sobre nazismo,Presidente encerrou visita a aliado Netanyahu no Museu do Holocausto e repetiu que nazismo é de esquerda declaração contrariada inclusive no site oficial do local,https://brasil.elpais.com/brasil/2019/04/02/internacional/1554216611_825972.html
242,A fortuna blindada de Cristiano Ronaldo,Atacante criou um império empresarial em cuja direção colocou sua família e um círculo muito íntimo de amigos,https://brasil.elpais.com/brasil/2019/03/22/deportes/1553256281_127660.html
210,“Refletir sobre nosso comportamento online é mais importante do que votar”,Pesquisador inglês Jamie Bartlett chama atenção para a cidadania na Internet:,https://brasil.elpais.com/brasil/2019/03/26/politica/1553628705_921854.html
18,Bolsonaro (des)governa o Brasil pelo Twitter,Ao tomar decisões pelo volume dos gritos nas redes sociais o presidente corrompe a democracia,https://brasil.elpais.com/brasil/2019/03/06/opinion/1551904505_351681.html
85,Presidente na ficção humorista é favorito em disputa eleitoral na Ucrânia,Admirador declarado de Bolsonaro o ator Volodymyr Zelenskiy lidera as pesquisas para as eleições deste domingo explorando a insatisfação popular com a elite política,https://brasil.elpais.com/brasil/2019/03/28/internacional/1553803929_078467.html
213,A tensão em escolas e universidades na esteira do massacre de Suzano,Instituições buscam formas para lidar com ameaças várias delas já identificadas pelas autoridades policiais como trote de adolescentes,https://brasil.elpais.com/brasil/2019/03/21/politica/1553207769_581816.html


Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention Instagram directly but mention social networks for instance:
* 229, 18, 242,  85, 151

In [19]:
true_docs = [229, 18, 242,  85, 151] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

query,top_n_plus,documents,precision,recall
[Instagram],0,"[85, 110, 151, 229, 242]",1.0,0.833333
"[Instagram, publicadas, contínuos, Gallup]",3,"[85, 151]",1.0,0.333333
"[Instagram, publicadas, contínuos, Gallup, sociais, tarifas]",5,"[210, 85, 151, 213, 150, 18, 19, 35, 204, 228]",0.3,0.500000
"[Instagram, publicadas, contínuos, Gallup, sociais, tarifas, exibe, aparece, totalmente, torna, jogam]",10,"[85, 151, 210, 229, 35, 110, 150, 213, 124, 18]",0.5,0.833333


* Overall expanding the query worsens the precision and does not improve the recall

<br>

## Expanded Queries for term 'real'

In [20]:
k = 10
term = 'real'

In [21]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10, N=n_docs)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str([term]), str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))

#### Result of *conjunctive retrieve by term* for 'real'

query,top_n_plus,documents
[real],0,"[213, 30, 85, 137, 188, 18, 25, 27, 36, 40]"
"[real, significa, justamente, realidade]",3,"[18, 3, 11, 98, 137, 151, 165, 30, 59, 124]"
"[real, significa, justamente, realidade, ficção, mostrar]",5,"[18, 30, 165, 11, 98, 137, 151, 183, 3, 59]"
"[real, significa, justamente, realidade, ficção, mostrar, perder, acreditam, salto, chama, vida]",10,"[165, 18, 30, 183, 11, 98, 151, 137, 148, 172]"


In [22]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

,title,subtitle
3,Quando os pais de Gabo perceberam que tinham um filho mentiroso,Gustavo Tatis percorre o universo de García Márquez e seus segredos de família intimamente ligados à obra que a Netflix adaptará para as telas
4,Rádios canadenses banem músicas de Michael Jackson da programação,Quebec Cogeco Media toma a decisão após queixas de ouvintes indignados com documentário sobre supostos abusos sexuais cometidos pelo cantor
11,As imagens questionadoras de Harun Farocki,Artista alemão protagoniza mostra no IMS em meio a debate sobre o papel da arte como crítica sociopolítica
12,O manuscrito de ‘Cem anos de solidão’ que García Márquez acreditava ter se perdido,México guarda cópia datilografada da grande obra do escritor colombiano.
15,A tragédia da Ucrânia,Anne Applebaum relata a fome premeditada por Stalin para subjugar a população da Ucrânia frear qualquer tentativa de nacionalismo e liquidar as organizações que resistiam a integrá-la à URSS
16,Por que o papa Francisco ordenou a abertura dos arquivos secretos do Vaticano sobre o Holocausto judeu?,Se Francisco realmente conseguir lançar luz sem esconder documentos em tudo o que ainda está oculto sobre o Holocausto e a Igreja passará à história
17,Um tuíte muito vulgar,Bolsonaro lança cortina de fumaça nas redes sociais em detrimento da imagem institucional do Brasil
18,Bolsonaro (des)governa o Brasil pelo Twitter,Ao tomar decisões pelo volume dos gritos nas redes sociais o presidente corrompe a democracia
19,Igualdade radical,Movimento feminista impõe uma nova energia democratizante
21,Taurus ascensão de uma fabricante de armas com Bolsonaro,Chegada do ultradireitista ao poder faz disparar as ações mas cifras não ofuscam crise profunda da empresa


Above we have a list of the documents that mention the term *real* directly

<br>

In [23]:
pd.set_option('max_colwidth', 200)
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

,title,subtitle,url
3,Quando os pais de Gabo perceberam que tinham um filho mentiroso,Gustavo Tatis percorre o universo de García Márquez e seus segredos de família intimamente ligados à obra que a Netflix adaptará para as telas,https://brasil.elpais.com/brasil/2019/03/06/cultura/1551888378_718292.html
137,Curto-circuito político faz da economia o calcanhar de Aquiles de Bolsonaro,Dificuldade de reverter o quadro é atribuída a vários fatores. Contas públicas no vermelho e contenção de gastos impedem que Governo impulsione a atividade econômica como aconteceu em outras crises,https://brasil.elpais.com/brasil/2019/03/21/economia/1553205979_786023.html
11,As imagens questionadoras de Harun Farocki,Artista alemão protagoniza mostra no IMS em meio a debate sobre o papel da arte como crítica sociopolítica,https://brasil.elpais.com/brasil/2019/03/12/cultura/1552400966_974298.html
18,Bolsonaro (des)governa o Brasil pelo Twitter,Ao tomar decisões pelo volume dos gritos nas redes sociais o presidente corrompe a democracia,https://brasil.elpais.com/brasil/2019/03/06/opinion/1551904505_351681.html
148,Bolsonaro: proibidão para menores e maiores,O preconceituoso capitão-presidente reivindica e cumpre o lugar de fala de um Brasil da pornochanchada,https://brasil.elpais.com/brasil/2019/03/11/opinion/1552309318_164942.html
151,Socialismo ‘millennial’ nos EUA,A temida palavra ‘socialista’ já não é mais tabu entre os jovens do país. Críticos da má distribuição de oportunidades e do ‘status quo’ encontraram sua voz em Alexandria Ocasio-Cortez,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551972909_806992.html
25,O breve discurso de Bolsonaro decepciona em Davos,Falta de detalhes sobre medidas concretas gera frustração na audiência do Fórum. “Foi um pouco decepcionante mas a verdade é que as metas que fixou foram alentadoras” avaliava o economista-chefe...,https://brasil.elpais.com/brasil/2019/01/22/economia/1548182020_953667.html
27,Puxado por Brasil e Espanha lucro do Santander cresce 18% em 2018,Integração com o Banco Popular e redução das provisões melhoram o balanço do banco com resultado positivo de 33 2 bilhões de reais,https://brasil.elpais.com/brasil/2019/01/30/economia/1548832025_256415.html
30,Matando como em ‘Call of Duty’,O assassino da Nova Zelândia viola a estética do videogame e cita um de seus maiores astros o youtuber Pew Die Pie para transformar sua matança em um espetáculo subjugante para um público jovem,https://brasil.elpais.com/brasil/2019/03/15/cultura/1552651637_070278.html
36,Os percalços dos planos A e B de Paulo Guedes para controlar o gasto público,Superministro propõe incluir sistema de capitalização em reforma da Previdência. Ele já pôs na mesa banir regras de gasto obrigatório com saúde e educação,https://brasil.elpais.com/brasil/2019/01/03/economia/1546549523_759922.html


Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'real' directly (e.g. talk about reality or brazilian currency):
* 3, 137, 151, 172, 27, 183, 188, 85

In [24]:
true_docs = [3, 137, 151, 172, 27, 183, 188, 85] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

query,top_n_plus,documents,precision,recall
[real],0,"[213, 30, 85, 137, 188, 18, 25, 27, 36, 40]",1.0,0.090909
"[real, significa, justamente, realidade]",3,"[18, 3, 11, 98, 137, 151, 165, 30, 59, 124]",1.0,0.090909
"[real, significa, justamente, realidade, ficção, mostrar]",5,"[18, 30, 165, 11, 98, 137, 151, 183, 3, 59]",1.0,0.090909
"[real, significa, justamente, realidade, ficção, mostrar, perder, acreditam, salto, chama, vida]",10,"[165, 18, 30, 183, 11, 98, 151, 137, 148, 172]",1.0,0.090909


* Expanding overall improves recall and worsens the precision
* Expanding with more than 3 terms improves recall and precision locally

<br>

## Expanded Queries for term 'reprodução'

In [25]:
k = 10
term = 'reprodução'

In [26]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10, N=n_docs)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str([term]), str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))

#### Result of *conjunctive retrieve by term* for 'reprodução'

query,top_n_plus,documents
[reprodução],0,"[160, 88, 123, 140, 196]"
"[reprodução, plataforma, programas, médicas]",3,"[196, 160, 123, 79, 109, 114, 11, 17, 41, 80]"
"[reprodução, plataforma, programas, médicas, costumes, música]",5,"[196, 160, 123, 109, 79, 114, 11, 17, 41, 80]"
"[reprodução, plataforma, programas, médicas, costumes, música, vazio, valem, usuário, preparação, bruto]",10,"[196, 123, 160, 109, 114, 79, 11, 17, 41, 80]"


In [27]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

,title,subtitle
88,Bruno Latour: “O sentimento de perder o mundo agora é coletivo”,Em seu último livro o influente pensador francês descreve um planeta onde a mudança climática altera tudo
123,França impulsiona o acesso juvenil à cultura com um cheque de 500 euros,Governo de Macron testa aplicativo com geolocalização para celular que convida aqueles que completam 18 anos a gastar em cinemas teatros livros cursos ou assinaturas ‘online’
140,Spotify lança publicidade apenas para pessoas felizes,A plataforma de música em ‘streaming’ estreia com a Renault sua primeira campanha de ‘happy targeting’ que só mostra os anúncios a quem ouve listas de reprodução mais alegres
160,Violência obstétrica uma forma de desumanização das mulheres,O fenômeno é muito mais comum do que a novidade da palavra parece sugerir: são muitas as mulheres que ignoram ter sofrido com isso
196,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais,Há dez anos uma total desconhecida tornou-se da noite para o dia a mulher mais famosa do mundo. É assim que ela vive hoje depois de conhecer a glória e a riqueza mas também a obscuridade


Above we have a list of the documents that mention the term *reprodução* directly

<br>

In [28]:
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

,title,subtitle,url
160,Violência obstétrica uma forma de desumanização das mulheres,O fenômeno é muito mais comum do que a novidade da palavra parece sugerir: são muitas as mulheres que ignoram ter sofrido com isso,https://brasil.elpais.com/brasil/2019/03/21/opinion/1553125734_101001.html
196,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais,Há dez anos uma total desconhecida tornou-se da noite para o dia a mulher mais famosa do mundo. É assim que ela vive hoje depois de conhecer a glória e a riqueza mas também a obscuridade,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html
41,Após acenar com embaixada Bolsonaro anuncia escritório comercial em Jerusalém,Presidente anuncia espaço para promoção comercial na cidade que tem peso estratégico para o Governo de Benjamin Netanyahu. Israel anuncia “forte apoio” à entrada do Brasil na OCDE,https://brasil.elpais.com/brasil/2019/03/31/politica/1554043546_030838.html
11,As imagens questionadoras de Harun Farocki,Artista alemão protagoniza mostra no IMS em meio a debate sobre o papel da arte como crítica sociopolítica,https://brasil.elpais.com/brasil/2019/03/12/cultura/1552400966_974298.html
140,Spotify lança publicidade apenas para pessoas felizes,A plataforma de música em ‘streaming’ estreia com a Renault sua primeira campanha de ‘happy targeting’ que só mostra os anúncios a quem ouve listas de reprodução mais alegres,https://brasil.elpais.com/brasil/2019/03/24/economia/1553429256_346859.html
109,Só sete jovens negros no Olimpo da educação pública de Nova York,O célebre colégio Stuyvesant se torna também o epicentro da segregação: apenas 7 dos 895 novos alunos para o próximo ano letivo são afro-americanos enquanto três quartos são asiáticos,https://brasil.elpais.com/brasil/2019/04/01/internacional/1554147846_545287.html
79,Dois sobreviventes do massacre de Parkland se suicidam na mesma semana,Comunidade escolar da Flórida nos EUA pede mais recursos para programas de saúde mental,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553626056_197921.html
80,Fracasso democrata para derrubar veto de Trump no Congresso,Câmara dos Representantes não consegue os dois terços de votos necessários para revogar a declaração de emergência do presidente para construção do muro com o México,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553633660_869823.html
17,Um tuíte muito vulgar,Bolsonaro lança cortina de fumaça nas redes sociais em detrimento da imagem institucional do Brasil,https://brasil.elpais.com/brasil/2019/03/07/opinion/1551926876_408228.html
114,Esportes vivem paralisia sem planos detalhados e nomeações travadas,Rebaixada de ministério a secretaria política esportiva de Bolsonaro espelha modelo do regime ditatorial com militares na linha de frente engessada pela falta de autonomia,https://brasil.elpais.com/brasil/2019/03/30/deportes/1553901938_175156.html


Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'reprodução' directly (e.g. talk about copying):
* 140, 160, 196, 88

In [29]:
true_docs = [140, 160, 196, 88] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

query,top_n_plus,documents,precision,recall
[reprodução],0,"[160, 88, 123, 140, 196]",1.0,1.0
"[reprodução, plataforma, programas, médicas]",3,"[196, 160, 123, 79, 109, 114, 11, 17, 41, 80]",0.3,0.6
"[reprodução, plataforma, programas, médicas, costumes, música]",5,"[196, 160, 123, 109, 79, 114, 11, 17, 41, 80]",0.3,0.6
"[reprodução, plataforma, programas, médicas, costumes, música, vazio, valem, usuário, preparação, bruto]",10,"[196, 123, 160, 109, 114, 79, 11, 17, 41, 80]",0.3,0.6


* Overall expanding the query worsens the precision 
* Recall is slightly improved when more than 3 elements are added

<br>

## Expanded Queries for term 'criativo'

In [30]:
k = 10
term = 'criativo'

In [31]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10, N=n_docs)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str([term]), str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))

#### Result of *conjunctive retrieve by term* for 'criativo'

query,top_n_plus,documents
[criativo],0,"[35, 150]"
"[criativo, cheque, conhecidas, acredito]",3,"[35, 150, 210, 25, 62, 68]"
"[criativo, cheque, conhecidas, acredito, mental, trabalhadores]",5,"[35, 150, 238, 79, 8, 22, 162, 196, 216, 225]"
"[criativo, cheque, conhecidas, acredito, mental, trabalhadores, parlamentares, efeitos, cara, podemos, acontecer]",10,"[35, 150, 238, 79, 216, 22, 162, 8, 196, 225]"


In [32]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

,title,subtitle
35,Finlândia conclui experiência de renda básica universal com resultados ambíguos,Durante dois anos 2.000 pessoas desempregadas receberam 2.370 reais por mês. Saúde autoestima e otimismo dos beneficiários melhoraram mas situação profissional pouco mudou
150,Quem mandou matar Marielle? E por quê?,Bolsonaro que governa o Brasil pela administração do ódio deveria ser o maior interessado em desvendar o crime


Above we have a list of the documents that mention the term *criativo* directly

<br>

In [33]:
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

,title,subtitle,url
225,Neto de Lula não morreu de meningite,Um mês depois da morte laudo de exames realizados pelo Instituto Adolfo Lutz descartam todos os tipos de meningite como causa do óbito de Arthur Araújo Lula da Silva de 7 anos,https://brasil.elpais.com/brasil/2019/04/02/politica/1554238552_750714.html
162,A era dos reptilianos,As teorias da conspiração pressupõem que os dois lados de uma disputa científica ou social devem ter a mesma veracidade,https://brasil.elpais.com/brasil/2019/03/21/opinion/1553190541_750395.html
35,Finlândia conclui experiência de renda básica universal com resultados ambíguos,Durante dois anos 2.000 pessoas desempregadas receberam 2.370 reais por mês. Saúde autoestima e otimismo dos beneficiários melhoraram mas situação profissional pouco mudou,https://brasil.elpais.com/brasil/2019/02/09/economia/1549710265_204922.html
68,Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”,Em entrevista ao EL PAÍS o ideólogo da extrema direita diz que o vice-primeiro-ministro italiano e o premiê húngaro Viktor Orbán são os políticos mais importantes da Europa atualmente,https://brasil.elpais.com/brasil/2019/03/24/internacional/1553454729_290547.html
196,O que a fama fez com Susan Boyle a mulher que revolucionou a música a televisão e as redes sociais,Há dez anos uma total desconhecida tornou-se da noite para o dia a mulher mais famosa do mundo. É assim que ela vive hoje depois de conhecer a glória e a riqueza mas também a obscuridade,https://brasil.elpais.com/brasil/2019/04/04/cultura/1554380845_480636.html
8,O lado mais sombrio de Dickens,Escritor e jornalista tentou internar sua mulher em um manicômio para poder se relacionar com atriz,https://brasil.elpais.com/brasil/2019/03/01/cultura/1551475506_900644.html
238,Saúde mental dos estudantes mais um desafio para as escolas brasileiras,Massacre de Suzano reabre debate sobre papel das instituições de ensino em identificar transtornos psiquiátricos. 80% dos alunos da rede pública com algum problema não recebem tratamento,https://brasil.elpais.com/brasil/2019/03/18/politica/1552928918_526670.html
79,Dois sobreviventes do massacre de Parkland se suicidam na mesma semana,Comunidade escolar da Flórida nos EUA pede mais recursos para programas de saúde mental,https://brasil.elpais.com/brasil/2019/03/26/internacional/1553626056_197921.html
210,“Refletir sobre nosso comportamento online é mais importante do que votar”,Pesquisador inglês Jamie Bartlett chama atenção para a cidadania na Internet:,https://brasil.elpais.com/brasil/2019/03/26/politica/1553628705_921854.html
150,Quem mandou matar Marielle? E por quê?,Bolsonaro que governa o Brasil pela administração do ódio deveria ser o maior interessado em desvendar o crime,https://brasil.elpais.com/brasil/2019/03/13/opinion/1552485039_897963.html


Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'criativo' directly (e.g. talk about creativity):
* 35, 210, 150

In [34]:
true_docs = [35, 210, 150] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

query,top_n_plus,documents,precision,recall
[criativo],0,"[35, 150]",1.0,0.666667
"[criativo, cheque, conhecidas, acredito]",3,"[35, 150, 210, 25, 62, 68]",0.5,1.000000
"[criativo, cheque, conhecidas, acredito, mental, trabalhadores]",5,"[35, 150, 238, 79, 8, 22, 162, 196, 216, 225]",0.2,0.666667
"[criativo, cheque, conhecidas, acredito, mental, trabalhadores, parlamentares, efeitos, cara, podemos, acontecer]",10,"[35, 150, 238, 79, 216, 22, 162, 8, 196, 225]",0.2,0.666667


* Expanding with the top 3 improves recall and worsens the precision
* Expanding with more than 3 terms consistently fails to improve the overall results 

<br>

## Expanded Queries for term 'hostil'

In [35]:
k = 10
term = 'hostil'

In [36]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10, N=n_docs)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str([term]), str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))

#### Result of *conjunctive retrieve by term* for 'hostil'

query,top_n_plus,documents
[hostil],0,"[94, 216]"
"[hostil, fere, delicado, judiciais]",3,"[94, 216, 168, 39, 75, 127, 149]"
"[hostil, fere, delicado, judiciais, Escola, Chegou]",5,"[94, 216, 5, 8, 44, 203, 243]"
"[hostil, fere, delicado, judiciais, Escola, Chegou, defendeu, causas, posse, Militar, sente]",10,"[94, 216, 8, 44, 5, 203, 243]"


In [37]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

,title,subtitle
5,Rosângela uma das crianças sequestradas por famílias de militares na ditadura,“Roubaram minha identidade” diz Rosângela Paraná cuja certidão de nascimento foi forjada pelo avô sargento.
91,O brasileiro insensato que rebate racismo com xenofobia,Jogador Serginho foi vítima de ofensas racistas e abandonou o campo na Bolívia. Por aqui alguns torcedores preferem ofender o país vizinho a refletir sobre seus próprios preconceitos
94,No Chile a política repudia a figura de Pinochet e matiza seu legado econômico,Piñera já fez vários gestos de rejeição ao passado do regime que matou mais de 3.000 chilenos. O legado econômico da ditadura também é contestado pela sociedade que cobra Educação e Previdência co...
216,Operando com 10% do orçamento Funai abandona postos e coordenações em áreas indígenas,Em zonas de conflito há coordenações que funcionam dentro de carros e funcionários que sofrem de esgotamento físico e mental por atuar sozinhos
245,Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo,Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino


Above we have a list of the documents that mention the term *hostil* directly

<br>

In [38]:
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

,title,subtitle,url
5,Rosângela uma das crianças sequestradas por famílias de militares na ditadura,“Roubaram minha identidade” diz Rosângela Paraná cuja certidão de nascimento foi forjada pelo avô sargento.,https://brasil.elpais.com/brasil/2019/03/29/politica/1553877780_122371.html
39,Maior elétrica dos EUA declara falência por responsabilidade nos incêndios na Califórnia,Pacific Gas & Electric enfrenta centenas de processos por negligência em incêndios com mortos.,https://brasil.elpais.com/brasil/2019/01/30/economia/1548804494_947066.html
168,Na Argentina falar da ditadura e dos militares que a conduziram é motivo de desonra,País teve um dos regimes mais sanguinários do continente com ao menos 30.000 mortos,https://brasil.elpais.com/brasil/2019/03/30/opinion/1553971198_297214.html
8,O lado mais sombrio de Dickens,Escritor e jornalista tentou internar sua mulher em um manicômio para poder se relacionar com atriz,https://brasil.elpais.com/brasil/2019/03/01/cultura/1551475506_900644.html
75,Crianças condenadas ao fogo e ao descaso,Creche em Janaúba Museu Nacional e os garotos do Ninho. Retratos de uma nação que queima o passado com a mesma passividade que destrói suas perspectivas de futuro,https://brasil.elpais.com/brasil/2019/02/21/deportes/1550784946_705165.html
44,Vinicius Junior decola no Real Madrid e chega à seleção,Agora referência do ataque merengue o jogador de 18 anos é o mais novo a ser convocado para o Brasil desde 2011,https://brasil.elpais.com/brasil/2019/02/26/deportes/1551205840_500748.html
203,A queda de outro ex-presidente do Brasil a nova fatura da Lava Jato à velha política brasileira,Ninguém podia imaginar em 2014 que a operação revelaria o maior escândalo de corrupção da história do Brasil e seria o estopim de mudanças profundas que se materializaram em um presidente de ultra...,https://brasil.elpais.com/brasil/2019/03/24/politica/1553385476_361260.html
243,Nanga Parbat a obsessão com a ‘montanha assassina’,A recente morte de dois alpinistas no topo mais letal depois do Annapurna acentua a angustiante luta com este cume há mais de um século,https://brasil.elpais.com/brasil/2019/03/20/deportes/1553098084_373740.html
149,Grupos que sustentam Maduro serão principal desafio a qualquer futuro governo na Venezuela,É erro grave presumir que queda de Maduro e promoção de eleições livres seriam garantia para resolver problemas do país vizinho,https://brasil.elpais.com/brasil/2019/03/11/opinion/1552342672_852167.html
216,Operando com 10% do orçamento Funai abandona postos e coordenações em áreas indígenas,Em zonas de conflito há coordenações que funcionam dentro de carros e funcionários que sofrem de esgotamento físico e mental por atuar sozinhos,https://brasil.elpais.com/brasil/2019/03/27/politica/1553700198_051341.html


Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'hostil' directly (e.g. talk about violence/hostility):
* 5, 168, 8, 75, 149, 216, 94

In [39]:
true_docs = [5, 168, 8, 75, 149, 216, 94] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

query,top_n_plus,documents,precision,recall
[hostil],0,"[94, 216]",1.000000,0.222222
"[hostil, fere, delicado, judiciais]",3,"[94, 216, 168, 39, 75, 127, 149]",0.714286,0.555556
"[hostil, fere, delicado, judiciais, Escola, Chegou]",5,"[94, 216, 5, 8, 44, 203, 243]",0.571429,0.444444
"[hostil, fere, delicado, judiciais, Escola, Chegou, defendeu, causas, posse, Militar, sente]",10,"[94, 216, 8, 44, 5, 203, 243]",0.571429,0.444444


* Expanding with the top 3 improves recall and worsens the precision
* Expanding with more than 3 terms consistently worsens the overall results 

## Summary

* `Expected Mutual Information Measure (EMIM)` gave the best results

* Expanding the query affected the results:
    * Worsened the `precision`
    * In some cases improved the `recall` 
        + Usually when adding the top 3, less often when adding the top 10